# Обучение модели Word2Vec на текстовых данных социальной сети VK, содержащих эмодзи

### Импорт необходимых библиотек

In [1]:
!pip install --proxy 10.0.0.1:3128 gensim pandas nltk pymorphy2 emoji

Using legacy 'setup.py install' for emoji, since package 'wheel' is not installed.
Using legacy 'setup.py install' for docopt, since package 'wheel' is not installed.
    Running setup.py install for docopt: started
    Running setup.py install for docopt: finished with status 'done'
    Running setup.py install for emoji: started
    Running setup.py install for emoji: finished with status 'done'


You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import nltk
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from gensim.models import Word2Vec as wv
from gensim.models import KeyedVectors
import pymorphy2
from pymorphy2 import MorphAnalyzer
import emoji
import numpy as np

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [6]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_ru.zip.


True

In [7]:
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\universal_tagset.zip.


True

### Подготовка датасета со смайликами

##### Чтение датасета 

In [8]:
emojis = pd.read_csv('datasets/marking.csv')

In [9]:
emojis = emojis.drop(columns=['image_id', 'name', 'group', 'sub_group', 'train'])

In [10]:
emojis

text    emoji
0                           Ухмыляющееся лицо        😀
1        Ухмыляющееся лицо с большими глазами        😃
2     Улыбающееся лицо с улыбающимися глазами        😄
3         Сияющее лицо с улыбающимися глазами        😁
4                 Ухмыляющееся щурящееся лицо        😆
...                                       ...      ...
2744                             Флаг: Замбия       🇲🇿
2745                           Флаг: Зимбабве       🇿🇼
2746                             Флаг: Англия  🏴󠁧󠁢󠁥󠁮󠁧󠁿
2747                          Флаг: Шотландия  🏴󠁧󠁢󠁳󠁣󠁴󠁿
2748                              Флаг: Уэльс  🏴󠁧󠁢󠁷󠁬󠁳󠁿

[2749 rows x 2 columns]

##### Токенизация описания эмодзи

In [11]:
regular = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
full_tokens = []
for text in emojis['text']:
  text = re.sub(regular, '', text)
  tokens = word_tokenize(text, language='russian')
  full_tokens.append(tokens)
emojis['tokens'] = full_tokens

In [12]:
emojis= emojis.drop('text', axis = 1)

In [13]:
emojis

emoji                                         tokens
0           😀                           [Ухмыляющееся, лицо]
1           😃     [Ухмыляющееся, лицо, с, большими, глазами]
2           😄  [Улыбающееся, лицо, с, улыбающимися, глазами]
3           😁      [Сияющее, лицо, с, улыбающимися, глазами]
4           😆                [Ухмыляющееся, щурящееся, лицо]
...       ...                                            ...
2744       🇲🇿                                 [Флаг, Замбия]
2745       🇿🇼                               [Флаг, Зимбабве]
2746  🏴󠁧󠁢󠁥󠁮󠁧󠁿                                 [Флаг, Англия]
2747  🏴󠁧󠁢󠁳󠁣󠁴󠁿                              [Флаг, Шотландия]
2748  🏴󠁧󠁢󠁷󠁬󠁳󠁿                                  [Флаг, Уэльс]

[2749 rows x 2 columns]

##### Приведение к нормальной форме всех слов в описании

In [14]:
filtered = []
morph = pymorphy2.MorphAnalyzer()
for tokens in emojis['tokens']:
  filtered_tokens = []
  for token in tokens:
    token = morph.normal_forms(token)[0]
    if token not in stopwords.words('russian'):
      filtered_tokens.append(token)
  filtered.append(filtered_tokens)

In [15]:
emojis['filtered'] = filtered

In [16]:
emojis = emojis.drop('tokens', axis = 1)

In [17]:
emojis

emoji                            filtered
0           😀                  [ухмыляться, лицо]
1           😃   [ухмыляться, лицо, больший, глаз]
2           😄  [улыбаться, лицо, улыбаться, глаз]
3           😁      [сиять, лицо, улыбаться, глаз]
4           😆        [ухмыляться, щуриться, лицо]
...       ...                                 ...
2744       🇲🇿                      [флаг, замбия]
2745       🇿🇼                    [флаг, зимбабве]
2746  🏴󠁧󠁢󠁥󠁮󠁧󠁿                      [флаг, англия]
2747  🏴󠁧󠁢󠁳󠁣󠁴󠁿                   [флаг, шотландия]
2748  🏴󠁧󠁢󠁷󠁬󠁳󠁿                       [флаг, уэльс]

[2749 rows x 2 columns]

##### Объединение смайлика с описанием для получения единого предложения для дальнейшего использования датасета для обучения модели

In [18]:
for e in emojis['emoji']:
  ind = emojis[emojis['emoji'] == e].index
  ind.to_list()
  emojis.at[ind[0], 'filtered'].append(e)

In [19]:
emojis = emojis.drop('emoji', axis = 1)

In [20]:
emojis

filtered
0                     [ухмыляться, лицо, 😀]
1      [ухмыляться, лицо, больший, глаз, 😃]
2     [улыбаться, лицо, улыбаться, глаз, 😄]
3         [сиять, лицо, улыбаться, глаз, 😁]
4           [ухмыляться, щуриться, лицо, 😆]
...                                     ...
2744                         [флаг, замбия]
2745                   [флаг, зимбабве, 🇿🇼]
2746                [флаг, англия, 🏴󠁧󠁢󠁥󠁮󠁧󠁿]
2747             [флаг, шотландия, 🏴󠁧󠁢󠁳󠁣󠁴󠁿]
2748                 [флаг, уэльс, 🏴󠁧󠁢󠁷󠁬󠁳󠁿]

[2749 rows x 1 columns]

### Подготовка датасета с комментариями ВК

##### Чтение датасета

In [21]:
vk_data = ['index', 'comment']

In [24]:
comments = pd.read_csv('datasets/dataset_vk.csv', names=colnames, header=0, sep='\t')

In [25]:
comments

index                                            comment
0          0                                          ['норма']
1          1                                              ['😁']
2          2                 ['воронка', 'цветок', 'вода', '💮']
3          3                                  ['сша', 'фашизм']
4          4              ['злодей', 'целый', 'месяц', 'ждать']
...      ...                                                ...
20575  20575                              ['с', 'фамилия', '🙄']
20576  20576  ['наш', 'день', 'повезти', 'мужик', 'фамилия',...
20577  20577               ['чудо', 'чудной', 'диво', 'дивный']
20578  20578  ['пригреться', 'уснуть', 'свариться', 'лягушка...
20579  20579  ['добрый', 'день', 'вполне', 'возможно', 'пере...

[20580 rows x 2 columns]

In [26]:
comments = comments.drop(columns = ['index'],axis = 1)

In [27]:
comments

comment
0                                              ['норма']
1                                                  ['😁']
2                     ['воронка', 'цветок', 'вода', '💮']
3                                      ['сша', 'фашизм']
4                  ['злодей', 'целый', 'месяц', 'ждать']
...                                                  ...
20575                              ['с', 'фамилия', '🙄']
20576  ['наш', 'день', 'повезти', 'мужик', 'фамилия',...
20577               ['чудо', 'чудной', 'диво', 'дивный']
20578  ['пригреться', 'уснуть', 'свариться', 'лягушка...
20579  ['добрый', 'день', 'вполне', 'возможно', 'пере...

[20580 rows x 1 columns]

## Загрузка датасета lenta.ru из corus (natasha)

In [28]:
lenta_data = pd.read_csv('datasets/lenta-ru-news.csv')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11212\1691548047.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  lenta_data = pd.read_csv('datasets/lenta-ru-news.csv')


In [29]:
lenta_data.head(2)

url  \
0   https://lenta.ru/news/1914/09/16/hungarnn/   
1  https://lenta.ru/news/1914/09/16/lermontov/   

                                               title  \
0  1914. Русские войска вступили в пределы Венгрии     
1  1914. Празднование столетия М.Ю. Лермонтова от...   

                                                text       topic  \
0  Бои у Сопоцкина и Друскеник закончились отступ...  Библиотека   
1  Министерство народного просвещения, в виду про...  Библиотека   

             tags        date  
0  Первая мировая  1914/09/16  
1  Первая мировая  1914/09/16

In [30]:
lenta_data.head(2)['text']

0    Бои у Сопоцкина и Друскеник закончились отступ...
1    Министерство народного просвещения, в виду про...
Name: text, dtype: object

In [31]:
lenta_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800975 entries, 0 to 800974
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     800975 non-null  object
 1   title   800975 non-null  object
 2   text    800970 non-null  object
 3   topic   738973 non-null  object
 4   tags    773756 non-null  object
 5   date    800975 non-null  object
dtypes: object(6)
memory usage: 36.7+ MB


In [32]:
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

def lemmatize(doc):
    tokens = []
    doc = re.sub(patterns, ' ', str(doc))
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    return tokens

In [69]:
df8 = pd.DataFrame()

In [70]:
df8['text'] = lenta_data['text'][700001:800000]

In [71]:
df8['text'] = df8['text'].apply(lemmatize)

In [66]:
df8['text'].head(5)

600001    [защитник, немецкий, «шальке», сборная, россия...
600002    [учёный, сша, обнаружить, атмосфера, юпитер, г...
600003    [россия, продолжать, сотрудничать, спецслужба,...
600004    [французский, модный, дом, выпустить, новый, м...
600005    [камера, наблюдение, зафиксировать, выяснение,...
Name: text, dtype: object

In [67]:
df8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 600001 to 699999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    99999 non-null  object
dtypes: object(1)
memory usage: 781.4+ KB


In [68]:
df8.to_csv('lenta_df8.csv', sep=',')